In [27]:
import csv
d={}
with open("data/legend.csv") as file_obj:
    file_data = csv.reader(file_obj)
    next(file_data)
    for row in file_data:
        key=row[2].lower()
        if key in d:
            d[key].append(row[1])
        else:
            d[key]=[row[1]]


In [28]:
emotion_list = list(d.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt']

In [29]:
import os
os.mkdir("Masterdir")
os.mkdir("Masterdir/Training")
os.mkdir("Masterdir/Testing")

for each in emotion_list:
    os.mkdir(os.path.join("Masterdir/Training/",each))
    os.mkdir(os.path.join("Masterdir/Testing/",each))

In [40]:
from shutil import copyfile
split_size = 0.8

for emotion, images in d.items():
  train_size = int(split_size*len(images))
  train_images = images[:train_size]
  test_images = images[train_size:]
  for image in train_images:
    source = os.path.join('images/', image)
    dest = os.path.join('Masterdir/Training/', emotion, image)
    copyfile(source, dest)
  for image in test_images:
    source = os.path.join('images/', image)
    dest = os.path.join('Masterdir/Testing/', emotion, image)
    copyfile(source, dest)

In [41]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [43]:
model = tf.keras.models.Sequential([
    Conv2D(16,(3,3),activation="relu", input_shape=(100,100,3)),
    MaxPooling2D(2, 2),
    Conv2D(32,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(1024,activation="relu"),
    Dense(8,activation="softmax")
])
model.compile(optimizer=Adam(lr=0.01),loss="categorical_crossentropy",metrics=["acc"])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 10, 10, 64)       0

In [44]:
training_dir = "Masterdir/Training"
testing_dir = "Masterdir/Testing"

train_datagen=ImageDataGenerator(rescale=0.1/255)
train_generator = train_datagen.flow_from_directory(
    training_dir,
    target_size=(100,100),
    class_mode = "categorical",
    batch_size=128
)

test_datagen = ImageDataGenerator(rescale=0.1/255)
test_generator = test_datagen.flow_from_directory(
    testing_dir,
    target_size = (100,100),
    class_mode= "categorical",
    batch_size = 128
)

Found 13683 images belonging to 8 classes.
Found 2742 images belonging to 8 classes.


In [45]:
es = EarlyStopping(monitor='val_acc', patience = 2, min_delta=0.01)

In [46]:
model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=train_generator,callbacks=[es])

C:\Users\visha\AppData\Local\Temp\ipykernel_13744\687556935.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator,epochs=10,verbose=1,validation_data=train_generator,callbacks=[es])


Epoch 1/10
107/107 [==============================] - 123s 1s/step - loss: 1.0191 - acc: 0.5435 - val_loss: 0.8440 - val_acc: 0.6922
Epoch 2/10
107/107 [==============================] - 69s 647ms/step - loss: 0.7405 - acc: 0.7373 - val_loss: 0.6770 - val_acc: 0.7666
Epoch 3/10
107/107 [==============================] - 69s 647ms/step - loss: 0.6293 - acc: 0.7821 - val_loss: 0.5856 - val_acc: 0.7948
Epoch 4/10
107/107 [==============================] - 71s 661ms/step - loss: 0.5849 - acc: 0.7976 - val_loss: 0.5338 - val_acc: 0.8164
Epoch 5/10
107/107 [==============================] - 69s 650ms/step - loss: 0.5303 - acc: 0.8163 - val_loss: 0.5018 - val_acc: 0.8262
Epoch 6/10
107/107 [==============================] - 69s 646ms/step - loss: 0.4970 - acc: 0.8261 - val_loss: 0.4658 - val_acc: 0.8414
Epoch 7/10
107/107 [==============================] - 69s 646ms/step - loss: 0.4641 - acc: 0.8371 - val_loss: 0.4469 - val_acc: 0.8451
Epoch 8/10
107/107 [==============================] - 68s